In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import complextorch as cvtorch
from torch.utils.data import DataLoader
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}")

import matplotlib.pyplot as plt
import json
import os
from collections import defaultdict
import random
import re

import antiglitch
from antiglitch import to_fd

Using device cuda


## Data processing

In [2]:
datadir = '/home/xangma/OneDrive/repos/antiglitch/data/'

In [3]:
# Load data
# files are named as follows: ifo-key-num.npz,
# e.g. L1-lowblip-0355.npz

# Find all ifos, keys, and numbers in the data directory
ifos = []
ml_models = []
numbers = []

# filename pattern: ifo-key-num.npz
filename_pattern = re.compile(r'([A-Z0-9]+)-([a-z]+)-(\d+).npz')

for file in os.listdir(datadir):
	# Get unique ifos:
	try:
		# check filename format
		if filename_pattern.match(file):
			ifo = file.split('-')[0]
			if ifo not in ifos:
				ifos.append(ifo)
			# Get unique keys:
			ml_model = file.split('-')[1]
			if ml_model not in ml_models:
				ml_models.append(ml_model)
	except:
		pass

datadict={}
for ifo in ifos:
	datadict[ifo] = {}
	for ml_model in ml_models:
		datadict[ifo][ml_model] = []
		for file in os.listdir(datadir):
			try:
				ifo_file = file.split('-')[0]
				key_file = file.split('-')[1]
				num = file.split('-')[2].split('.')[0]
				if ifo_file == ifo and key_file == ml_model:
					datadict[ifo][ml_model].append(int(num))
			except:
				pass

In [4]:
# load results
resultsjson=datadir + 'all_PE_v3.json'
with open(resultsjson, 'r') as f:
	results = json.load(f)

In [5]:
# load all glitches into an array using the datadict and Snippet class
glitches = {}
for ifo in ifos:
    if ifo not in glitches:
        glitches[ifo] = {}
    for ml_model in ml_models:
        if ml_model not in glitches[ifo]:
            glitches[ifo][ml_model] = {}
        for num in datadict[ifo][ml_model]:
            # try:
                if num not in glitches[ifo][ml_model]:
                    glitches[ifo][ml_model][num] = {}
                snip = antiglitch.SnippetNormed(ifo, ml_model, num, datadir)
                glitches[ifo][ml_model][num]['data'] = to_fd(snip.whts)
                glitches[ifo][ml_model][num]['invasd'] = snip.invasd
            # except:
            #     pass

/home/xangma/OneDrive/repos/antiglitch/antiglitch/utils.py:29: RuntimeWarning: divide by zero encountered in power
  invasd = ((4096.*npz['psd'])**-0.5)[:4097]


In [6]:
ml_label_map = {'Blip_Low_Frequency':'lowblip', 'Blip':'blip', 'Koi_Fish':'koi', 'Tomte':'tomte'}
results_keys = ['f0', 'f0_sd', 'gbw', 'gbw_sd', 'amp_r', 'amp_r_sd', 'amp_i', 'amp_i_sd', 'time', 'time_sd', 'num']

nums = list(results['num'].values())
for i in range(len(nums)):
    try:
        stri = str(i)
        ifo = results['ifo'][stri]
        ml_label = ml_label_map[results['ml_label'][stri]]
        
        for key in results_keys:
            glitches[ifo][ml_label][nums[i]][key] = results[key][stri]
    except:
        pass

In [7]:
# Step 1: Restructure the dataset (Yes I know it was originally in this format, you leave me alone)
distributions = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for ifo in glitches:
    for ml_model in glitches[ifo]:
        for glitch_num in glitches[ifo][ml_model]:
            for key in results_keys:
                distributions[ifo][ml_model][key].append(glitches[ifo][ml_model][glitch_num][key])

In [8]:
def gen_sample(ifo, ml_model, tosample=['f0', 'gbw', 'amp_r', 'amp_i', 'time']):
    res = {key:None for key in tosample}
    for key in tosample:
        draw = np.random.choice(distributions[ifo][ml_model][key])
        draw_sd = distributions[ifo][ml_model][key + '_sd'][distributions[ifo][ml_model][key].index(draw)]
        draw_final = np.random.normal(draw, draw_sd)
        res[key] = draw_final
    return res

def new_init_Snippet(self, invasd):
        self.invasd = invasd

SnippetNormed = type('SnippetNormed', (antiglitch.SnippetNormed,), {'__init__': new_init_Snippet})


In [9]:
def get_snip(ifos, ml_models, tosample=['f0', 'gbw', 'amp_r', 'amp_i', 'time']):
		ifo = np.random.choice(ifos)
		ml_model = np.random.choice(ml_models)
		glitch_num = random.choice(datadict[ifo][ml_model])
		glitch_invasd = glitches[ifo][ml_model][glitch_num]['invasd']
		snip = SnippetNormed(glitch_invasd)
		inf = gen_sample(ifo, ml_model)
		inf['freqs'] = snip.invasd
		snip.set_infer(inf)
		x = snip.fglitch
		# check for nans
		if np.isnan(x).any():
			return get_snip(ifos, ml_models, tosample)
		y = (snip.inf['f0'], snip.inf['gbw'])
		return x, y

In [49]:
# define dataset
NTRAIN = 200000
NTEST = 50000
cachedataset_prefix = "antiglitch_cvnn_dataset_"

class GlitchDataset(torch.utils.data.TensorDataset):
	def __init__(self, ifos, ml_models, datadict, glitches, distributions, size):
		self.ifos = ifos
		self.ml_models = ml_models
		self.datadict = datadict
		self.glitches = glitches
		self.distributions = distributions
		self.size = size
		self.x_arr = []
		self.y_arr = []
		# create dataset
		if cachedataset_prefix + str(size) + '.npz' in os.listdir(datadir):
			data = np.load(datadir + cachedataset_prefix + str(size) + '.npz')
			self.x_arr = data['x_arr']
			self.y_arr = data['y_arr']
		else:
			for i in range(size):
				x, y = get_snip(ifos, ml_models)
				self.x_arr.append(x)
				self.y_arr.append(y)
			self.x_arr = np.array(self.x_arr)
			self.y_arr = np.array(self.y_arr)
			# normalize x_arr
			real_parts = np.real(self.x_arr)
			imag_parts = np.imag(self.x_arr)
			
			mean_real = np.mean(real_parts)
			std_real = np.std(real_parts)
			
			mean_imag = np.mean(imag_parts)
			std_imag = np.std(imag_parts)
			
			normalized_real = (real_parts - mean_real) / std_real
			normalized_imag = (imag_parts - mean_imag) / std_imag
			
			self.x_arr = normalized_real + 1j * normalized_imag
			# normalize y_arr
			y1_mean = self.y_arr[:,0].mean()
			y1_std = self.y_arr[:,0].std()
			y2_mean = self.y_arr[:,1].mean()
			y2_std = self.y_arr[:,1].std()
			self.y_arr[:,0] = (self.y_arr[:,0] - y1_mean)/y1_std
			self.y_arr[:,1] = (self.y_arr[:,1] - y2_mean)/y2_std
			np.savez(datadir + cachedataset_prefix + str(size) + '.npz', x_arr=self.x_arr, y_arr=self.y_arr)
		# convert to tensors
		self.x_arr = torch.tensor(self.x_arr, dtype=torch.complex64, device=device)
		self.y_arr = torch.tensor(self.y_arr, dtype=torch.float32, device=device)

	def __len__(self):
		return self.size

	def __getitem__(self, idx):
		return self.x_arr[idx], self.y_arr[idx]

train_data = GlitchDataset(ifos, ml_models, datadict, glitches, distributions, NTRAIN)
test_data = GlitchDataset(ifos, ml_models, datadict, glitches, distributions, NTEST)
testitem = train_data.__getitem__(0)
print(testitem[0][1:20], testitem[1])

tensor([ 0.0036-3.9260e-04j, -0.1319-7.6944e-02j, -0.6444-4.6743e-01j,
        -1.2034-1.0866e+00j, -1.3743-1.5321e+00j, -1.0950-1.5078e+00j,
        -0.7373-1.2664e+00j, -0.4926-1.0779e+00j, -0.2991-8.6897e-01j,
        -0.1503-6.2775e-01j, -0.0614-4.4096e-01j, -0.0128-3.1188e-01j,
         0.0128-2.2166e-01j,  0.0259-1.6265e-01j,  0.0308-1.1464e-01j,
         0.0298-7.5746e-02j,  0.0277-5.1892e-02j,  0.0256-3.6700e-02j,
         0.0222-2.4451e-02j], device='cuda:0') tensor([-0.3419,  0.5125], device='cuda:0')


In [11]:
# These were cells used for preprocessing the data before I was told I could use the Snippet class to generate - whoops.
# # Now flatten to x and y data where x is glitched data and y is the [f0, gbw]

# x = []
# y = []
# for ifo in ifos:
# 	for key in keys:
# 		for num in datadict[ifo][key]:
# 			try:
# 				x.append(glitches[ifo][key][num]['data'])
# 				y.append([glitches[ifo][key][num]['f0'], glitches[ifo][key][num]['gbw']])
# 			except:
# 				pass
# x = np.array(x)
# y = np.array(y)
# type(x), x.shape, type(y), y.shape
# # Split data into training and testing

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# # Convert to torch tensors
# x_train = torch.tensor(x_train, dtype=torch.complex64)
# x_test = torch.tensor(x_test, dtype=torch.complex64)
# y_train = torch.tensor(y_train, dtype=torch.float32)
# y_test = torch.tensor(y_test, dtype=torch.float32)
# # Normalize data
# x_train = (x_train - x_train.mean()) / x_train.std()
# x_test = (x_test - x_test.mean()) / x_test.std()
# y_train = (y_train - y_train.mean()) / y_train.std()
# y_test = (y_test - y_test.mean()) / y_test.std()
# # put data on device
# x_train = x_train.to(device)
# x_test = x_test.to(device)
# y_train = y_train.to(device)
# y_test = y_test.to(device)
# # Print out a description and sample of the data
# print(x_train.shape, x_train.dtype, x_train[0])
# print(x_test.shape, x_test.dtype, x_test[0])
# print(y_train.shape, y_train.dtype, y_train[0])
# print(y_test.shape, y_test.dtype, y_test[0])
# train_data = torch.utils.data.TensorDataset(x_train, y_train)
# test_data = torch.utils.data.TensorDataset(x_test, y_test)

In [26]:
# Load into DataLoader
train_loader = DataLoader(dataset=train_data, batch_size=1024, shuffle=True, drop_last=False)
test_loader = DataLoader(dataset=test_data, batch_size=1024, shuffle=False, drop_last=False)
list(train_loader.__iter__())[0][0].shape

torch.Size([1024, 513])

## Model Definition

In [22]:
class ComplexValuedNN(nn.Module):
    def __init__(self):
        super(ComplexValuedNN, self).__init__()

        # Complex Conv Layer 1
        self.conv1 = cvtorch.nn.CVConv1d(1, 32, 3, padding="same")
        self.activation1 = cvtorch.nn.zReLU()
        # self.bn1 = cvtorch.nn.CVBatchNorm1d(32)
        # self.dropout1 = cvtorch.nn.CVDropout(0.25)

        # Complex Conv Layer 2
        self.conv2 = cvtorch.nn.CVConv1d(32, 64, 5, padding="same")
        self.activation2 = cvtorch.nn.zReLU()
        # self.bn2 = cvtorch.nn.CVBatchNorm1d(64)
        # self.dropout2 = cvtorch.nn.CVDropout(0.25)

        # Fully connected Layer 1
        self.fc1 = cvtorch.nn.CVLinear(64 * 513, 128)
        self.activation3 = cvtorch.nn.zReLU()
        # self.bn3 = cvtorch.nn.CVBatchNorm1d(1024)
        # self.dropout3 = cvtorch.nn.CVDropout(0.25)

        # Fully connected Layer 2
        # self.fc2 = cvtorch.nn.CVLinear(1024, 1024)
        # self.activation4 = cvtorch.nn.zReLU()
        # self.bn4 = cvtorch.nn.CVBatchNorm1d(1024)
        # self.dropout4 = cvtorch.nn.CVDropout(0.25)
        # Fully connected Layer 2
        self.fc3 = cvtorch.nn.CVLinear(128, 32)
        self.activation5 = cvtorch.nn.zReLU()
        
        # self.bn5 = cvtorch.nn.CVBatchNorm1d(32)
        # self.dropout5 = cvtorch.nn.CVDropout(0.25)
        
        # Output Layer (real-valued)
        self.output_layer = nn.Linear(32 * 2, 2)

    def forward(self, x):
        # Add channel dimension for convolution
        x = x.unsqueeze(1)

        # Conv Layer 1
        x = self.conv1(x)
        x = self.activation1(x)
        # x = self.bn1(x)
        # x = self.dropout1(x)

        # # # Conv Layer 2
        x = self.conv2(x)
        x = self.activation2(x)
        # x = self.bn2(x)
        # x = self.dropout2(x)

        # Flatten the tensor
        x = x.view(x.size(0), -1)

        # Fully Connected Layer 1
        x = self.fc1(x)
        x = self.activation3(x)
        # x = self.bn3(x)
        # x = self.dropout3(x)

        # Fully Connected Layer 2
        # x = self.fc2(x)
        # x = self.activation4(x)
        # x = self.bn4(x)
        # x = self.dropout4(x)
        x = self.fc3(x)
        x = self.activation5(x)
        # x = self.bn5(x)
        # x = self.dropout5(x)
        # Transform complex valued output for the final real-valued layer
        real_x = torch.cat((x.real, x.imag), dim=1)  # Merge real and imag parts

        # Output Layer
        out = self.output_layer(real_x)

        return out

In [23]:
# Define the input and output dimensions based on the dataset
input_dim = 513  # Number of input features
output_dim = 2  # Number of output units (as specified in the dataset)

# Create an instance of the network
model = ComplexValuedNN()

# put model on device
model.to(device)

# model = torch.compile(model)
# Print the model architecture
print(model)
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")

ComplexValuedNN(
  (conv1): CVConv1d(
    (conv_r): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=same)
    (conv_i): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=same)
  )
  (activation1): zReLU()
  (conv2): CVConv1d(
    (conv_r): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=same)
    (conv_i): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=same)
  )
  (activation2): zReLU()
  (fc1): CVLinear(
    (linear_r): Linear(in_features=32832, out_features=128, bias=False)
    (linear_i): Linear(in_features=32832, out_features=128, bias=False)
  )
  (activation3): zReLU()
  (fc3): CVLinear(
    (linear_r): Linear(in_features=128, out_features=32, bias=False)
    (linear_i): Linear(in_features=128, out_features=32, bias=False)
  )
  (activation5): zReLU()
  (output_layer): Linear(in_features=64, out_features=2, bias=True)
)
Number of parameters: 8434178


In [15]:
# Load model if it exists
model_path = 'complex_nn_model.pth'
try:
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print('Model loaded')
except:
    print('Model not loaded')

## Training

In [27]:
# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=20, verbose=True)

In [28]:
# Training loop
torch.autograd.set_detect_anomaly(True)
num_epochs = 200
for epoch in range(num_epochs):
    model.train()

    for inputs, targets in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    

    # Test the model
    model.eval()
    with torch.no_grad():
        total_loss = 0
        for inputs, targets in test_loader:
            outputs = model(inputs)
            total_loss += criterion(outputs, targets).item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {total_loss/len(test_loader):.4f}")
    
    # update the learning rate
    scheduler.step(total_loss/len(test_loader))

    writer.add_scalars("Loss", {"Train": loss.item(), "Validation": total_loss/len(test_loader)}, epoch)
    writer.add_scalar("Learning rate", optimizer.param_groups[0]['lr'], epoch)
    
    # save the model every 10 epochs
    if epoch % 10 == 0:
        torch.save(model.state_dict(), model_path)
        
writer.flush()


Epoch [1/200], Training Loss: 0.6597, Validation Loss: 1.3385
Epoch [2/200], Training Loss: 0.3960, Validation Loss: 1.5106
Epoch [3/200], Training Loss: 0.3573, Validation Loss: 1.8187
Epoch [4/200], Training Loss: 0.4524, Validation Loss: 1.7916
Epoch [5/200], Training Loss: 0.3311, Validation Loss: 1.4534
Epoch [6/200], Training Loss: 0.3834, Validation Loss: 1.7292
Epoch [7/200], Training Loss: 0.3286, Validation Loss: 2.0100
Epoch [8/200], Training Loss: 0.2803, Validation Loss: 2.1725
Epoch [9/200], Training Loss: 0.3617, Validation Loss: 2.1758
Epoch [10/200], Training Loss: 0.2700, Validation Loss: 1.9132
Epoch [11/200], Training Loss: 0.2453, Validation Loss: 1.8925
Epoch [12/200], Training Loss: 0.1639, Validation Loss: 0.3369
Epoch [13/200], Training Loss: 0.2377, Validation Loss: 0.3165
Epoch [14/200], Training Loss: 0.2353, Validation Loss: 0.2952
Epoch [15/200], Training Loss: 0.1965, Validation Loss: 0.2633
Epoch [16/200], Training Loss: 0.1690, Validation Loss: 0.2594
E

KeyboardInterrupt: 

In [48]:
if type(outputs) == torch.Tensor:
    outputs = outputs.cpu().detach().numpy()
if type(targets) == torch.Tensor:
    targets = targets.cpu().detach().numpy()
for i in range(0,10):
    print(f"Prediction: {outputs[i]}")
    print(f"Target: {targets[i]}")
    print("\n")

Prediction: [ 0.11066131 -0.82846916]
Target: [ 0.2531437  -0.76916826]


Prediction: [ 2.4060965 -0.9313154]
Target: [ 3.581295  -0.7205246]


Prediction: [-0.28759983  0.22473028]
Target: [-0.38967282  0.13980909]


Prediction: [-0.33022934  1.7921206 ]
Target: [-0.3106424  2.2877557]


Prediction: [-0.47873828 -0.14912502]
Target: [-0.4583077  -0.07159861]


Prediction: [-0.62700087  2.3018472 ]
Target: [-0.51930755  2.738316  ]


Prediction: [-0.5536676  0.7405968]
Target: [-0.49190506  0.8744962 ]


Prediction: [-0.5106758  1.113033 ]
Target: [-0.35114104  1.5091016 ]


Prediction: [-0.38125008 -0.7338947 ]
Target: [-0.5385673  -0.79498357]


Prediction: [-0.39388457  2.105945  ]
Target: [-0.52113533  1.5611671 ]




In [ ]:
# Save model
model_path = 'complex_nn_model.pth'
torch.save(model.state_dict(), model_path)

In [ ]:
# Load model
model = ComplexValuedNN(input_dim, output_dim)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>